# Day 2 (0-2h): 공정 최적화 & 품질 예측

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# 필요 라이브러리 설치
!pip install -q numpy pandas matplotlib seaborn
!pip install -q scikit-learn xgboost lightgbm
!pip install -q shap  # SHAP 해석
!pip install -q optuna  # 하이퍼파라미터 최적화
!pip install -q scipy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# ML 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb

# 설명가능 AI
import shap

# 최적화
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')

## 2. DoE 기초: 공정 파라미터 최적화 실험 설계

In [ ]:
class DesignOfExperiments:
    """실험계획법(DoE) 구현"""
    
    def __init__(self):
        self.factors = {
            'temperature': {'low': 150, 'high': 200, 'unit': '°C'},
            'pressure': {'low': 100, 'high': 150, 'unit': 'bar'},
            'speed': {'low': 1000, 'high': 1500, 'unit': 'rpm'},
            'time': {'low': 30, 'high': 60, 'unit': 'min'}
        }
        
    def full_factorial_design(self, n_levels=2):
        """완전요인설계 생성"""
        import itertools
        
        factors_list = list(self.factors.keys())
        levels = []
        
        for factor in factors_list:
            if n_levels == 2:
                levels.append([self.factors[factor]['low'], 
                              self.factors[factor]['high']])
            else:
                # 3수준 이상
                low = self.factors[factor]['low']
                high = self.factors[factor]['high']
                levels.append(np.linspace(low, high, n_levels).tolist())
        
        # 모든 조합 생성
        experiments = list(itertools.product(*levels))
        
        # DataFrame으로 변환
        doe_df = pd.DataFrame(experiments, columns=factors_list)
        
        return doe_df
    
    def fractional_factorial_design(self, resolution='III'):
        """부분요인설계 생성"""
        # 2^(4-1) fractional factorial design
        # Resolution III: 주효과는 명확, 2차 상호작용과 혼동
        
        design_matrix = [
            [-1, -1, -1, +1],
            [+1, -1, -1, -1],
            [-1, +1, -1, -1],
            [+1, +1, -1, +1],
            [-1, -1, +1, -1],
            [+1, -1, +1, +1],
            [-1, +1, +1, +1],
            [+1, +1, +1, -1]
        ]
        
        # 코드화된 값을 실제 값으로 변환
        experiments = []
        for row in design_matrix:
            exp = []
            for i, (factor, levels) in enumerate(self.factors.items()):
                if row[i] == -1:
                    exp.append(levels['low'])
                else:
                    exp.append(levels['high'])
            experiments.append(exp)
        
        doe_df = pd.DataFrame(experiments, columns=list(self.factors.keys()))
        
        return doe_df
    
    def central_composite_design(self):
        """중심합성설계 (CCD) 생성"""
        # 2^k factorial + 2k axial points + center points
        k = len(self.factors)  # 인자 수
        alpha = np.sqrt(k)  # 축점 거리
        
        experiments = []
        factors_list = list(self.factors.keys())
        
        # Factorial points (corners of cube)
        for i in range(2**k):
            exp = []
            for j in range(k):
                if (i >> j) & 1:
                    exp.append(self.factors[factors_list[j]]['high'])
                else:
                    exp.append(self.factors[factors_list[j]]['low'])
            experiments.append(exp)
        
        # Axial points (star points)
        for j in range(k):
            # +alpha point
            exp_plus = []
            exp_minus = []
            for i in range(k):
                center = (self.factors[factors_list[i]]['low'] + 
                         self.factors[factors_list[i]]['high']) / 2
                if i == j:
                    range_val = (self.factors[factors_list[i]]['high'] - 
                                self.factors[factors_list[i]]['low']) / 2
                    exp_plus.append(center + alpha * range_val / 2)
                    exp_minus.append(center - alpha * range_val / 2)
                else:
                    exp_plus.append(center)
                    exp_minus.append(center)
            experiments.append(exp_plus)
            experiments.append(exp_minus)
        
        # Center points
        center_point = [(self.factors[f]['low'] + self.factors[f]['high']) / 2 
                       for f in factors_list]
        for _ in range(3):  # 3 center point replicates
            experiments.append(center_point.copy())
        
        doe_df = pd.DataFrame(experiments, columns=factors_list)
        
        return doe_df
    
    def simulate_response(self, doe_df):
        """응답 시뮬레이션 (실제로는 실험 수행)"""
        # 가상의 품질 응답 함수
        # Y = 50 + 0.5*T - 0.3*P + 0.4*S + 0.2*t + noise
        # + interaction terms + quadratic terms
        
        responses = []
        
        for _, row in doe_df.iterrows():
            T = (row['temperature'] - 175) / 25  # 정규화
            P = (row['pressure'] - 125) / 25
            S = (row['speed'] - 1250) / 250
            t = (row['time'] - 45) / 15
            
            # 주효과
            y = 85 + 5*T - 3*P + 4*S + 2*t
            
            # 상호작용
            y += 1.5*T*P - 1*T*S + 0.8*P*S
            
            # 2차 항
            y -= 2*T**2 - 1.5*P**2 - 1*S**2
            
            # 노이즈
            y += np.random.normal(0, 2)
            
            responses.append(y)
        
        doe_df['quality'] = responses
        
        return doe_df

# DoE 실행
doe = DesignOfExperiments()

# 1. 완전요인설계 (2^4 = 16 실험)
full_factorial = doe.full_factorial_design(n_levels=2)
full_factorial = doe.simulate_response(full_factorial)

print("Full Factorial Design (2^4)")
print(f"Number of experiments: {len(full_factorial)}")
print(full_factorial.head())
print(f"\nQuality range: {full_factorial['quality'].min():.1f} - {full_factorial['quality'].max():.1f}")

# 2. 부분요인설계 (2^(4-1) = 8 실험)
fractional_factorial = doe.fractional_factorial_design()
fractional_factorial = doe.simulate_response(fractional_factorial)

print("\n" + "="*50)
print("Fractional Factorial Design (2^(4-1))")
print(f"Number of experiments: {len(fractional_factorial)}")
print(fractional_factorial.head())

# 3. 중심합성설계
ccd = doe.central_composite_design()
ccd = doe.simulate_response(ccd)

print("\n" + "="*50)
print("Central Composite Design")
print(f"Number of experiments: {len(ccd)}")
print(f"Quality range: {ccd['quality'].min():.1f} - {ccd['quality'].max():.1f}")

## 3. 앙상블 모델링: GBM/RandomForest + SHAP 해석

In [ ]:
class QualityPredictionModels:
    """품질 예측 앙상블 모델"""
    
    def __init__(self):
        self.models = {
            'RandomForest': RandomForestRegressor(
                n_estimators=100,
                max_depth=10,
                random_state=42
            ),
            'GradientBoosting': GradientBoostingRegressor(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=5,
                random_state=42
            ),
            'XGBoost': xgb.XGBRegressor(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=5,
                random_state=42
            ),
            'LightGBM': lgb.LGBMRegressor(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=5,
                random_state=42,
                verbose=-1
            )
        }
        self.scaler = StandardScaler()
        self.feature_importance = {}
        
    def prepare_data(self, data, target_col='quality'):
        """데이터 준비"""
        X = data.drop(columns=[target_col])
        y = data[target_col]
        
        # Train/Test 분할
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        # 스케일링
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)
        
        return X_train_scaled, X_test_scaled, y_train, y_test, X.columns
    
    def train_and_evaluate(self, X_train, X_test, y_train, y_test, feature_names):
        """모델 학습 및 평가"""
        results = {}
        
        for name, model in self.models.items():
            # 학습
            model.fit(X_train, y_train)
            
            # 예측
            y_pred_train = model.predict(X_train)
            y_pred_test = model.predict(X_test)
            
            # 평가 메트릭
            train_mse = mean_squared_error(y_train, y_pred_train)
            test_mse = mean_squared_error(y_test, y_pred_test)
            train_mae = mean_absolute_error(y_train, y_pred_train)
            test_mae = mean_absolute_error(y_test, y_pred_test)
            train_r2 = r2_score(y_train, y_pred_train)
            test_r2 = r2_score(y_test, y_pred_test)
            
            # 교차 검증
            cv_scores = cross_val_score(
                model, X_train, y_train, cv=5, 
                scoring='neg_mean_squared_error'
            )
            
            results[name] = {
                'model': model,
                'train_mse': train_mse,
                'test_mse': test_mse,
                'train_mae': train_mae,
                'test_mae': test_mae,
                'train_r2': train_r2,
                'test_r2': test_r2,
                'cv_mse': -cv_scores.mean(),
                'cv_std': cv_scores.std(),
                'predictions': y_pred_test
            }
            
            # Feature importance
            if hasattr(model, 'feature_importances_'):
                self.feature_importance[name] = pd.DataFrame({
                    'feature': feature_names,
                    'importance': model.feature_importances_
                }).sort_values('importance', ascending=False)
        
        return results
    
    def ensemble_predict(self, models_dict, X_test):
        """앙상블 예측 (평균)"""
        predictions = []
        
        for name, result in models_dict.items():
            model = result['model']
            pred = model.predict(X_test)
            predictions.append(pred)
        
        # 평균 앙상블
        ensemble_pred = np.mean(predictions, axis=0)
        
        return ensemble_pred

# 더 많은 데이터 생성 (CCD 사용)
large_doe = doe.central_composite_design()
# 추가 샘플 생성
for _ in range(100):
    random_exp = pd.DataFrame([{
        'temperature': np.random.uniform(150, 200),
        'pressure': np.random.uniform(100, 150),
        'speed': np.random.uniform(1000, 1500),
        'time': np.random.uniform(30, 60)
    }])
    large_doe = pd.concat([large_doe, random_exp], ignore_index=True)

large_doe = doe.simulate_response(large_doe)

# 모델 학습
qpm = QualityPredictionModels()
X_train, X_test, y_train, y_test, feature_names = qpm.prepare_data(large_doe)

print("Training ensemble models...")
results = qpm.train_and_evaluate(X_train, X_test, y_train, y_test, feature_names)

# 성능 비교
performance_df = pd.DataFrame({
    model: {
        'Train R²': res['train_r2'],
        'Test R²': res['test_r2'],
        'Train MAE': res['train_mae'],
        'Test MAE': res['test_mae'],
        'CV MSE': res['cv_mse']
    }
    for model, res in results.items()
}).T

print("\n" + "="*60)
print("MODEL PERFORMANCE COMPARISON")
print("="*60)
print(performance_df.round(3))

# 앙상블 예측
ensemble_pred = qpm.ensemble_predict(results, X_test)
ensemble_mse = mean_squared_error(y_test, ensemble_pred)
ensemble_r2 = r2_score(y_test, ensemble_pred)

print(f"\nEnsemble Model Performance:")
print(f"  Test MSE: {ensemble_mse:.3f}")
print(f"  Test R²: {ensemble_r2:.3f}")

## 4. SHAP을 활용한 모델 해석

In [ ]:
# SHAP 분석
print("Calculating SHAP values...")

# XGBoost 모델 선택 (SHAP과 호환성 좋음)
best_model = results['XGBoost']['model']

# SHAP explainer 생성
explainer = shap.Explainer(best_model, X_train)
shap_values = explainer(X_test)

# Feature importance plot
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_test, feature_names=feature_names, show=False)
plt.title('SHAP Feature Importance')
plt.tight_layout()
plt.show()

# Waterfall plot for single prediction
plt.figure(figsize=(10, 6))
shap.waterfall_plot(shap_values[0], show=False)
plt.title('SHAP Waterfall Plot - Single Prediction')
plt.tight_layout()
plt.show()

# Feature interactions
for i, feature in enumerate(feature_names[:2]):
    plt.figure(figsize=(8, 5))
    shap.scatter_plot(shap_values[:, i], color=shap_values, show=False)
    plt.title(f'SHAP Dependence Plot - {feature}')
    plt.tight_layout()
    plt.show()

## 5. SPC (Statistical Process Control) 구현

In [ ]:
class StatisticalProcessControl:
    """통계적 공정 관리 (SPC)"""
    
    def __init__(self):
        self.control_limits = {}
        self.process_capability = {}
        
    def calculate_control_limits(self, data, n_sigma=3):
        """관리한계 계산 (X-bar, R charts)"""
        mean = np.mean(data)
        std = np.std(data, ddof=1)
        
        ucl = mean + n_sigma * std  # Upper Control Limit
        lcl = mean - n_sigma * std  # Lower Control Limit
        
        self.control_limits = {
            'UCL': ucl,
            'CL': mean,
            'LCL': lcl,
            'sigma': std
        }
        
        return self.control_limits
    
    def check_control_rules(self, data):
        """Western Electric 규칙 검사"""
        violations = []
        ucl = self.control_limits['UCL']
        lcl = self.control_limits['LCL']
        cl = self.control_limits['CL']
        sigma = self.control_limits['sigma']
        
        # Rule 1: 1 point beyond 3σ
        rule1 = np.where((data > ucl) | (data < lcl))[0]
        if len(rule1) > 0:
            violations.append({'rule': 1, 'indices': rule1.tolist(),
                             'description': 'Point beyond 3σ limits'})
        
        # Rule 2: 2 out of 3 consecutive points beyond 2σ
        for i in range(len(data) - 2):
            subset = data[i:i+3]
            beyond_2sigma = np.sum((subset > cl + 2*sigma) | 
                                  (subset < cl - 2*sigma))
            if beyond_2sigma >= 2:
                violations.append({'rule': 2, 'indices': [i, i+1, i+2],
                                 'description': '2 of 3 points beyond 2σ'})
        
        # Rule 3: 4 out of 5 consecutive points beyond 1σ
        for i in range(len(data) - 4):
            subset = data[i:i+5]
            beyond_1sigma = np.sum((subset > cl + sigma) | 
                                  (subset < cl - sigma))
            if beyond_1sigma >= 4:
                violations.append({'rule': 3, 'indices': list(range(i, i+5)),
                                 'description': '4 of 5 points beyond 1σ'})
        
        # Rule 4: 8 consecutive points on one side of centerline
        for i in range(len(data) - 7):
            subset = data[i:i+8]
            if np.all(subset > cl) or np.all(subset < cl):
                violations.append({'rule': 4, 'indices': list(range(i, i+8)),
                                 'description': '8 points on one side'})
        
        return violations
    
    def calculate_process_capability(self, data, usl, lsl):
        """공정능력지수 계산 (Cp, Cpk)"""
        mean = np.mean(data)
        std = np.std(data, ddof=1)
        
        # Cp: 공정능력지수 (규격 폭 / 공정 변동)
        cp = (usl - lsl) / (6 * std)
        
        # Cpk: 치우침을 고려한 공정능력지수
        cpu = (usl - mean) / (3 * std)
        cpl = (mean - lsl) / (3 * std)
        cpk = min(cpu, cpl)
        
        # Pp, Ppk: 장기 공정능력
        pp = cp  # 단기와 동일하게 계산 (실제로는 장기 표준편차 사용)
        ppk = cpk
        
        # 불량률 예측 (ppm)
        z_usl = (usl - mean) / std
        z_lsl = (lsl - mean) / std
        ppm_upper = (1 - stats.norm.cdf(z_usl)) * 1e6
        ppm_lower = stats.norm.cdf(z_lsl) * 1e6
        ppm_total = ppm_upper + ppm_lower
        
        self.process_capability = {
            'Cp': cp,
            'Cpk': cpk,
            'Pp': pp,
            'Ppk': ppk,
            'PPM_total': ppm_total,
            'PPM_upper': ppm_upper,
            'PPM_lower': ppm_lower
        }
        
        return self.process_capability
    
    def plot_control_chart(self, data, title='Control Chart'):
        """관리도 그리기"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
        
        # X-bar chart
        ax1.plot(data, 'o-', markersize=4, linewidth=1, color='blue')
        ax1.axhline(y=self.control_limits['UCL'], color='red', 
                   linestyle='--', label='UCL')
        ax1.axhline(y=self.control_limits['CL'], color='green', 
                   linestyle='-', label='CL')
        ax1.axhline(y=self.control_limits['LCL'], color='red', 
                   linestyle='--', label='LCL')
        
        # 2σ lines
        ax1.axhline(y=self.control_limits['CL'] + 2*self.control_limits['sigma'], 
                   color='orange', linestyle=':', alpha=0.5)
        ax1.axhline(y=self.control_limits['CL'] - 2*self.control_limits['sigma'], 
                   color='orange', linestyle=':', alpha=0.5)
        
        # 1σ lines
        ax1.axhline(y=self.control_limits['CL'] + self.control_limits['sigma'], 
                   color='yellow', linestyle=':', alpha=0.5)
        ax1.axhline(y=self.control_limits['CL'] - self.control_limits['sigma'], 
                   color='yellow', linestyle=':', alpha=0.5)
        
        ax1.set_ylabel('Measurement')
        ax1.set_title(f'{title} - X-bar Chart')
        ax1.legend(loc='upper right')
        ax1.grid(True, alpha=0.3)
        
        # Moving Range chart
        mr = np.abs(np.diff(data))
        mr_mean = np.mean(mr)
        mr_ucl = mr_mean * 3.267  # D4 for n=2
        
        ax2.plot(mr, 'o-', markersize=4, linewidth=1, color='purple')
        ax2.axhline(y=mr_ucl, color='red', linestyle='--', label='UCL')
        ax2.axhline(y=mr_mean, color='green', linestyle='-', label='CL')
        ax2.axhline(y=0, color='red', linestyle='--', label='LCL')
        
        ax2.set_xlabel('Sample')
        ax2.set_ylabel('Moving Range')
        ax2.set_title('Moving Range Chart')
        ax2.legend(loc='upper right')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# SPC 시뮬레이션
# 생산 데이터 시뮬레이션 (100개 샘플)
np.random.seed(42)
production_data = np.random.normal(85, 3, 100)

# 일부 이상 패턴 주입
production_data[30:35] = np.random.normal(92, 2, 5)  # 상한 이탈
production_data[60:68] = np.random.normal(87, 1, 8)  # 한쪽 치우침

# SPC 분석
spc = StatisticalProcessControl()

# 관리한계 계산
control_limits = spc.calculate_control_limits(production_data[:30])  # 초기 30개로 한계 설정
print("Control Limits:")
for key, value in control_limits.items():
    print(f"  {key}: {value:.2f}")

# 규칙 위반 검사
violations = spc.check_control_rules(production_data)
print(f"\nFound {len(violations)} rule violations")
for v in violations[:3]:  # 처음 3개만 출력
    print(f"  Rule {v['rule']}: {v['description']} at indices {v['indices'][:5]}...")

# 공정능력 계산 (USL=95, LSL=75)
process_cap = spc.calculate_process_capability(production_data, usl=95, lsl=75)
print("\nProcess Capability:")
for key, value in process_cap.items():
    if 'PPM' in key:
        print(f"  {key}: {value:.1f}")
    else:
        print(f"  {key}: {value:.3f}")

# 관리도 그리기
spc.plot_control_chart(production_data, title='Quality Characteristic')

## 6. EWMA와 기타 관리도

In [ ]:
class AdvancedControlCharts:
    """고급 관리도 (EWMA, CUSUM)"""
    
    def ewma_chart(self, data, lambda_param=0.2, L=3):
        """EWMA (Exponentially Weighted Moving Average) 관리도"""
        n = len(data)
        ewma = np.zeros(n)
        
        # 초기값
        ewma[0] = data[0]
        
        # EWMA 계산
        for i in range(1, n):
            ewma[i] = lambda_param * data[i] + (1 - lambda_param) * ewma[i-1]
        
        # 관리한계 계산
        mean = np.mean(data[:30])  # 초기 데이터로 평균 추정
        std = np.std(data[:30], ddof=1)
        
        # EWMA 관리한계 (시간에 따라 변함)
        ucl = np.zeros(n)
        lcl = np.zeros(n)
        
        for i in range(n):
            variance_factor = np.sqrt(lambda_param / (2 - lambda_param) * 
                                     (1 - (1 - lambda_param)**(2*(i+1))))
            ucl[i] = mean + L * std * variance_factor
            lcl[i] = mean - L * std * variance_factor
        
        return ewma, ucl, lcl, mean
    
    def cusum_chart(self, data, target, k=0.5, h=5):
        """CUSUM (Cumulative Sum) 관리도"""
        n = len(data)
        cusum_pos = np.zeros(n)
        cusum_neg = np.zeros(n)
        
        std = np.std(data[:30], ddof=1)
        
        for i in range(1, n):
            cusum_pos[i] = max(0, data[i] - (target + k*std) + cusum_pos[i-1])
            cusum_neg[i] = max(0, (target - k*std) - data[i] + cusum_neg[i-1])
        
        # 결정한계
        h_line = h * std
        
        return cusum_pos, cusum_neg, h_line
    
    def plot_advanced_charts(self, data, title='Advanced Control Charts'):
        """고급 관리도 시각화"""
        fig, axes = plt.subplots(3, 1, figsize=(12, 10))
        
        # 원본 데이터
        axes[0].plot(data, 'o-', markersize=3, linewidth=1, color='blue', alpha=0.6)
        axes[0].set_ylabel('Original Data')
        axes[0].set_title(f'{title} - Original Data')
        axes[0].grid(True, alpha=0.3)
        
        # EWMA Chart
        ewma, ucl, lcl, mean = self.ewma_chart(data)
        axes[1].plot(ewma, 'o-', markersize=3, linewidth=1, color='green', label='EWMA')
        axes[1].plot(ucl, 'r--', label='UCL')
        axes[1].plot(lcl, 'r--', label='LCL')
        axes[1].axhline(y=mean, color='gray', linestyle='-', alpha=0.5, label='Target')
        axes[1].set_ylabel('EWMA Value')
        axes[1].set_title('EWMA Control Chart (λ=0.2)')
        axes[1].legend(loc='upper right')
        axes[1].grid(True, alpha=0.3)
        
        # CUSUM Chart
        target = np.mean(data[:30])
        cusum_pos, cusum_neg, h_line = self.cusum_chart(data, target)
        axes[2].plot(cusum_pos, 'b-', linewidth=1.5, label='C+ (Upper)')
        axes[2].plot(-cusum_neg, 'r-', linewidth=1.5, label='C- (Lower)')
        axes[2].axhline(y=h_line, color='red', linestyle='--', alpha=0.5, label='Decision Limit')
        axes[2].axhline(y=-h_line, color='red', linestyle='--', alpha=0.5)
        axes[2].axhline(y=0, color='gray', linestyle='-', alpha=0.3)
        axes[2].set_xlabel('Sample')
        axes[2].set_ylabel('CUSUM Value')
        axes[2].set_title('CUSUM Control Chart')
        axes[2].legend(loc='upper right')
        axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# 고급 관리도 적용
acc = AdvancedControlCharts()
acc.plot_advanced_charts(production_data, title='Quality Monitoring')

# EWMA vs Standard Chart 민감도 비교
print("\nControl Chart Sensitivity Comparison:")
print("- Standard X-bar chart: Detects large shifts quickly")
print("- EWMA chart: Better for detecting small sustained shifts")
print("- CUSUM chart: Excellent for detecting small shifts, accumulates evidence")

## 실습 과제

1. **실험계획 최적화**: 자신의 공정에 맞는 DoE 설계 및 최적 조건 탐색
2. **앙상블 모델 개선**: Stacking, Voting 등 고급 앙상블 기법 적용
3. **실시간 SPC**: 스트리밍 데이터에서 작동하는 실시간 관리도 구현
4. **다변량 관리도**: Hotelling T² 차트 등 다변량 공정 모니터링 구현